# Parse Supplemental Stats Forms


In [1]:
%pip install requests pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import json
import requests
import pandas as pd
from pathlib import Path
from typing import Dict, Iterable, List

FORM_FILES = {
    'en': Path('Copy of 2024-2025 Supplemental Statistical Report.xlsx'),
    'fr': Path('data/2024-2025 Rapport statistique supplémentaire.xlsx'),
}

KEY_SHEET = 'ATIP_ForConsumption'


In [3]:
def load_key_sheet(path: Path) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=KEY_SHEET, header=None)
    df = df.rename(columns={0: 'id', 2: 'subsection_number'})
    df = df[df['id'].astype(str).str.match(r'^Q\d+R\d+C\d+$')].copy()
    df['subsection_number'] = df['subsection_number'].astype(str)
    return df


def extract_subsection_labels(path: Path, locale: str) -> Dict[str, str]:
    df = pd.read_excel(path, sheet_name=0, header=None)
    labels: Dict[str, str] = {}
    for val in df.values.flatten():
        if not isinstance(val, str):
            continue
        val = val.strip()
        match = re.match(r'^(\d+\.\d+)\s+(.+)$', val)
        if match:
            labels[match.group(1)] = match.group(2).strip()

    sin_label = None
    foreign_label = None
    for val in df.values.flatten():
        if not isinstance(val, str):
            continue
        if locale == 'en':
            if 'SIN' in val:
                sin_label = val.strip()
            if 'foreign nationals' in val:
                foreign_label = val.strip()
        else:
            if 'NAS' in val:
                sin_label = val.strip()
            if 'ressortissants étrangers' in val:
                foreign_label = val.strip()

    if sin_label:
        labels['3.0'] = sin_label
    if foreign_label:
        labels['4.0'] = foreign_label

    return labels


def build_records_from_key(df: pd.DataFrame, labels: Dict[str, str], locale: str) -> List[Dict[str, str]]:
    records = []
    section_name = f'section_name_{locale}'
    subsection_name = f'subsection_name_{locale}'
    title_name = f'title_{locale}'

    for _, row in df.iterrows():
        subsection_number = str(row['subsection_number']).strip()
        section_number = subsection_number.split('.')[0] if subsection_number else None
        records.append({
            'id': row['id'],
            'section_number': section_number,
            section_name: f"Section {section_number}" if section_number else None,
            'subsection_number': subsection_number,
            subsection_name: labels.get(subsection_number),
            title_name: None,
        })

    return records


def expand_supplemental_ids(records: List[Dict[str, str]]) -> List[Dict[str, str]]:
    parsed = []
    for record in records:
        match = re.match(r'^Q(\d+)R(\d+)C(\d+)$', record['id'])
        if not match:
            continue
        parsed.append((int(match.group(1)), int(match.group(2)), int(match.group(3)), record))

    groups: Dict[int, Dict[str, int]] = {}
    for q, row, col, _ in parsed:
        data = groups.setdefault(q, {'max_row': 0, 'max_col': 0})
        data['max_row'] = max(data['max_row'], row)
        data['max_col'] = max(data['max_col'], col)

    existing = {r['id'] for r in records}
    updated = list(records)
    for q, info in groups.items():
        max_row = info['max_row']
        max_col = info['max_col']
        if max_row == 10 and max_col == 2:
            target_rows = 11
            target_cols = 3
        elif max_row == 10 and max_col == 1:
            target_rows = 11
            target_cols = 1
        else:
            target_rows = max_row
            target_cols = max_col

        template = next((r for r in records if r['id'].startswith(f'Q{q}R')), None)
        if not template:
            continue

        for row in range(1, target_rows + 1):
            for col in range(1, target_cols + 1):
                new_id = f'Q{q}R{row}C{col}'
                if new_id in existing:
                    continue
                new_record = dict(template)
                new_record['id'] = new_id
                updated.append(new_record)
                existing.add(new_id)

    return updated


def merge_languages(df_en: pd.DataFrame, df_fr: pd.DataFrame) -> pd.DataFrame:
    combined = pd.merge(df_en, df_fr, on='id', how='outer', suffixes=('_en', '_fr'))
    combined['section_number'] = combined['section_number_en'].fillna(combined['section_number_fr'])
    combined['subsection_number'] = combined['subsection_number_en'].fillna(combined['subsection_number_fr'])

    final_cols = [
        'id',
        'section_number',
        'section_name_en',
        'section_name_fr',
        'subsection_number',
        'subsection_name_en',
        'subsection_name_fr',
        'title_en',
        'title_fr',
    ]

    return combined[final_cols]


def add_header_fields(df: pd.DataFrame) -> pd.DataFrame:
    header_rows = [
        {
            'id': 'NameOfInstitution',
            'section_number': None,
            'section_name_en': 'General Information',
            'section_name_fr': 'Informations générales',
            'subsection_number': None,
            'subsection_name_en': 'Identification',
            'subsection_name_fr': 'Identification',
            'title_en': None,
            'title_fr': None,
        },
        {
            'id': 'ReportingPeriodStart',
            'section_number': None,
            'section_name_en': 'Reporting period',
            'section_name_fr': "Période d'établissement de rapport",
            'subsection_number': None,
            'subsection_name_en': 'Start Date',
            'subsection_name_fr': 'Date de début',
            'title_en': None,
            'title_fr': None,
        },
        {
            'id': 'ReportingPeriodEnd',
            'section_number': None,
            'section_name_en': 'Reporting period',
            'section_name_fr': "Période d'établissement de rapport",
            'subsection_number': None,
            'subsection_name_en': 'End Date',
            'subsection_name_fr': 'Date de fin',
            'title_en': None,
            'title_fr': None,
        },
    ]
    header_df = pd.DataFrame(header_rows)
    return pd.concat([header_df, df], ignore_index=True)


def custom_id_sort_key(id_str: str):
    if id_str == 'NameOfInstitution':
        return (-3, -3, -3)
    if id_str == 'ReportingPeriodStart':
        return (-2, -2, -2)
    if id_str == 'ReportingPeriodEnd':
        return (-1, -1, -1)

    match = re.match(r'^Q(\d+)R(\d+)C(\d+)$', id_str or '')
    if match:
        q_num = int(match.group(1))
        row_num = int(match.group(2))
        col_num = int(match.group(3))
        if q_num in (1, 3):
            return (q_num, col_num, row_num)
        return (q_num, row_num, col_num)

    return (float('inf'), float('inf'), float('inf'))


def sort_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    sorted_df = df.copy()
    sorted_df['sort_key'] = sorted_df['id'].apply(custom_id_sort_key)
    sorted_df = sorted_df.sort_values(by='sort_key').drop(columns='sort_key').reset_index(drop=True)
    return sorted_df


def export_to_excel(df: pd.DataFrame, filename: str = 'combined_supplemental_form_data.xlsx') -> None:
    df.to_excel(filename, index=False)
    print(f"DataFrame successfully exported to '{filename}'")


In [4]:
key_en = load_key_sheet(FORM_FILES['en'])
key_fr = load_key_sheet(FORM_FILES['fr'])

labels_en = extract_subsection_labels(FORM_FILES['en'], 'en')
labels_fr = extract_subsection_labels(FORM_FILES['fr'], 'fr')

records_en = expand_supplemental_ids(build_records_from_key(key_en, labels_en, 'en'))
records_fr = expand_supplemental_ids(build_records_from_key(key_fr, labels_fr, 'fr'))

df_en = pd.DataFrame(records_en)
df_fr = pd.DataFrame(records_fr)

combined = merge_languages(df_en, df_fr)
combined = add_header_fields(combined)
combined = sort_dataframe(combined)

display(combined.head(10))
export_to_excel(combined)


,id,section_number,section_name_en,section_name_fr,subsection_number,subsection_name_en,subsection_name_fr,title_en,title_fr
0,NameOfInstitution,None,General Information,Informations générales,None,Identification,Identification,None,None
1,ReportingPeriodStart,None,Reporting period,Période d'établissement de rapport,None,Start Date,Date de début,None,None
2,ReportingPeriodEnd,None,Reporting period,Période d'établissement de rapport,None,End Date,Date de fin,None,None
3,Q1R1C1,1,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,None,None
4,Q1R2C1,1,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,None,None
5,Q1R3C1,1,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,None,None
6,Q1R4C1,1,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,None,None
7,Q1R5C1,1,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,None,None
8,Q1R6C1,1,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,None,None
9,Q1R7C1,1,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,None,None


DataFrame successfully exported to 'combined_supplemental_form_data.xlsx'


## Convert published dataset URL to long CSV


In [5]:
import json
import pandas as pd
from pathlib import Path
import unicodedata
import re
import requests
from difflib import SequenceMatcher


def _normalize_name(name: str) -> str:
    if not isinstance(name, str):
        return ''
    text = unicodedata.normalize('NFKD', name).casefold()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[‐‑‒–—―]', '-', text)
    return text.strip()


def _similarity(a: str, b: str) -> float:
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a, b).ratio()


def load_gc_org_lookup(org_json_url: str) -> pd.DataFrame:
    if org_json_url.startswith('http'):
        resp = requests.get(org_json_url)
        resp.raise_for_status()
        data = resp.json()
    else:
        data = json.loads(Path(org_json_url).read_text(encoding='utf-8'))

    records = data.get('records') or data.get('result', {}).get('records')
    fields = data.get('fields') or data.get('result', {}).get('fields')
    if not records or not fields:
        raise ValueError('Unexpected org JSON structure; missing fields/records')

    cols = [f['id'] for f in fields]
    df = pd.DataFrame(records, columns=cols)

    keep_cols = [
        'gc_orgID',
        'harmonized_name',
        'nom_harmonise',
        'legal_title',
        'appellation_legale',
        'preferred_name',
        'nom_prefere',
    ]
    for c in keep_cols:
        if c not in df.columns:
            df[c] = None
    df_lookup = df[keep_cols].copy()

    def collect_variants(row):
        variants = [
            row['harmonized_name'],
            row['nom_harmonise'],
            row['legal_title'],
            row['appellation_legale'],
            row['preferred_name'],
            row['nom_prefere'],
        ]
        norms = {_normalize_name(v) for v in variants if _normalize_name(v)}
        return list(norms)

    df_lookup['norm_variants'] = df_lookup.apply(collect_variants, axis=1)
    return df_lookup


def match_institution(name: str, lookup: pd.DataFrame, similarity_threshold: float = 0.88):
    norm = _normalize_name(name)
    if not norm:
        return pd.Series({'gc_orgID': None, 'institution_en': None, 'institution_fr': None})

    exact = lookup[lookup['norm_variants'].apply(lambda vs: norm in vs)]
    if not exact.empty:
        r = exact.iloc[0]
        return pd.Series({
            'gc_orgID': r['gc_orgID'],
            'institution_en': r['harmonized_name'],
            'institution_fr': r['nom_harmonise'],
        })

    best_row = None
    best_ratio = 0.0
    for _, cand in lookup.iterrows():
        for variant in cand['norm_variants']:
            ratio = _similarity(norm, variant)
            if ratio > best_ratio:
                best_ratio = ratio
                best_row = cand
    if best_row is not None and best_ratio >= similarity_threshold:
        return pd.Series({
            'gc_orgID': best_row['gc_orgID'],
            'institution_en': best_row['harmonized_name'],
            'institution_fr': best_row['nom_harmonise'],
        })

    return pd.Series({'gc_orgID': None, 'institution_en': name, 'institution_fr': None})


def dataset_url_to_long_csv(dataset_url: str,
                            reporting_start: str,
                            reporting_end: str,
                            output_csv: str = 'long_from_url.csv',
                            wide_path: str = 'combined_supplemental_form_data.xlsx',
                            org_json_url: str = 'https://open.canada.ca/data/en/datastore/dump/cb5b5566-f599-4d12-abae-8279a0230928?format=json') -> pd.DataFrame:
    # Download a wide Supplemental dataset and convert to long CSV aligned to the form schema with GC org IDs.
    wide = pd.read_excel(wide_path)

    value_rows = wide[~wide['id'].isin(['NameOfInstitution', 'ReportingPeriodStart', 'ReportingPeriodEnd'])].reset_index(drop=True)

    raw = pd.read_excel(dataset_url, header=None)
    expected_cols = len(value_rows) + 1  # institution + values
    if raw.shape[1] != expected_cols:
        raise ValueError(f"Column mismatch: raw has {raw.shape[1]} columns, expected {expected_cols} (institution + form fields)")

    data_rows = raw.iloc[3:].reset_index(drop=True)

    lookup = load_gc_org_lookup(org_json_url)

    start_date = pd.to_datetime(reporting_start)
    end_date = pd.to_datetime(reporting_end)

    frames = []
    for _, row in data_rows.iterrows():
        institution_raw = str(row.iloc[0]).strip()
        if not institution_raw:
            continue

        values = row.iloc[1:]
        if len(values) != len(value_rows):
            raise ValueError(f"Value length mismatch for {institution_raw}: {len(values)} vs {len(value_rows)}")

        match = match_institution(institution_raw, lookup)

        frame = value_rows.copy()
        frame['gc_orgID'] = match['gc_orgID']
        frame['institution_en'] = match['institution_en']
        frame['institution_fr'] = match['institution_fr']
        frame['ReportingPeriodStart'] = start_date
        frame['ReportingPeriodEnd'] = end_date
        frame['value'] = values.values
        frames.append(frame)

    if not frames:
        raise ValueError('No institution rows found in dataset.')

    long_df = pd.concat(frames, ignore_index=True)

    col_order = [
        'gc_orgID',
        'institution_en',
        'institution_fr',
        'ReportingPeriodStart',
        'ReportingPeriodEnd',
        'id',
        'section_number',
        'section_name_en',
        'section_name_fr',
        'subsection_number',
        'subsection_name_en',
        'subsection_name_fr',
        'title_en',
        'title_fr',
        'value',
    ]

    long_df = long_df[col_order]
    long_df.to_csv(output_csv, index=False)
    print(f"Wrote {len(long_df)} rows for {data_rows.shape[0]} institutions to {output_csv}")
    return long_df


In [6]:
DATASETS = [
    {
        'url': 'https://open.canada.ca/data/dataset/236294e1-bc74-486f-ab97-422227bc8832/resource/1124ca1d-2d69-4921-a759-79f960e84561/download/supplemental-dataset-2023-24.xlsx',
        'start': '2023-04-01',
        'end': '2024-03-31',
        'output': 'combined_supplemental_form_data_long_2023_24.csv',
    },
    {
        'url': 'https://open.canada.ca/data/dataset/236294e1-bc74-486f-ab97-422227bc8832/resource/d812b0b9-320f-465c-8c20-66ef90b76db5/download/supplemental-dataset-2024-25.xlsx',
        'start': '2024-04-01',
        'end': '2025-03-31',
        'output': 'combined_supplemental_form_data_long_2024_25.csv',
    },
]

for cfg in DATASETS:
    dataset_url_to_long_csv(
        cfg['url'],
        reporting_start=cfg['start'],
        reporting_end=cfg['end'],
        output_csv=cfg['output'],
    )


Wrote 17550 rows for 195 institutions to combined_supplemental_form_data_long_2023_24.csv
Wrote 17820 rows for 198 institutions to combined_supplemental_form_data_long_2024_25.csv


## Build consolidated Supplemental CSV


In [7]:
import pandas as pd
from pathlib import Path


def build_consolidated_csv(files=None, output='Supplemental-AI_refactored.csv'):
    # Concat multiple long CSVs into one file.
    if files is None:
        files = [
            'combined_supplemental_form_data_long_2023_24.csv',
            'combined_supplemental_form_data_long_2024_25.csv',
        ]

    frames = []
    missing = []
    for f in files:
        p = Path(f)
        if p.exists():
            frames.append(pd.read_csv(p))
        else:
            missing.append(f)

    if missing:
        raise FileNotFoundError(f"Missing input CSVs: {missing}")
    if not frames:
        raise ValueError('No input files provided.')

    combined = pd.concat(frames, ignore_index=True)
    combined.to_csv(output, index=False)
    print(f"Wrote {output} with {len(combined)} rows from {len(frames)} files.")
    return combined


In [8]:
build_consolidated_csv()


Wrote Supplemental-AI_refactored.csv with 35370 rows from 2 files.


,gc_orgID,institution_en,institution_fr,ReportingPeriodStart,ReportingPeriodEnd,id,section_number,section_name_en,section_name_fr,subsection_number,subsection_name_en,subsection_name_fr,title_en,title_fr,value
0,2297.0,Administrative Tribunals Support Service of Ca...,Service canadien d'appui aux tribunaux adminis...,2023-04-01,2024-03-31,Q1R1C1,1.0,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,NaN,NaN,4
1,2297.0,Administrative Tribunals Support Service of Ca...,Service canadien d'appui aux tribunaux adminis...,2023-04-01,2024-03-31,Q1R2C1,1.0,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,NaN,NaN,0
2,2297.0,Administrative Tribunals Support Service of Ca...,Service canadien d'appui aux tribunaux adminis...,2023-04-01,2024-03-31,Q1R3C1,1.0,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,NaN,NaN,0
3,2297.0,Administrative Tribunals Support Service of Ca...,Service canadien d'appui aux tribunaux adminis...,2023-04-01,2024-03-31,Q1R4C1,1.0,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,NaN,NaN,0
4,2297.0,Administrative Tribunals Support Service of Ca...,Service canadien d'appui aux tribunaux adminis...,2023-04-01,2024-03-31,Q1R5C1,1.0,Section 1,Section 1,1.1,Requests carried over to next reporting period...,Demandes reportées à la prochaine période d'ét...,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35365,NaN,Yukon Surface Rights Board,NaN,2024-04-01,2025-03-31,Q4R9C1,2.0,Section 2,Section 2,2.2,Active complaints with the Privacy Commissione...,Plaintes actives auprès du Commissaire à la pr...,NaN,NaN,0
35366,NaN,Yukon Surface Rights Board,NaN,2024-04-01,2025-03-31,Q4R10C1,2.0,Section 2,Section 2,2.2,Active complaints with the Privacy Commissione...,Plaintes actives auprès du Commissaire à la pr...,NaN,NaN,0
35367,NaN,Yukon Surface Rights Board,NaN,2024-04-01,2025-03-31,Q4R11C1,2.0,Section 2,Section 2,2.2,Active complaints with the Privacy Commissione...,Plaintes actives auprès du Commissaire à la pr...,NaN,NaN,0
35368,NaN,Yukon Surface Rights Board,NaN,2024-04-01,2025-03-31,Q5R1C1,3.0,Section 3,Section 3,3.0,Has your institution begun a new collection or...,Votre institution a-t-elle commencé une nouvel...,NaN,NaN,1


## Generate populated Supplemental form as HTML


In [ ]:
# %pip install ipywidgets openpyxl
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from pathlib import Path
from bs4 import BeautifulSoup
from http.server import ThreadingHTTPServer, SimpleHTTPRequestHandler
import functools
import socket
import threading
import webbrowser
import openpyxl
import re

WET_HEAD_HTML = """
<link href="https://wet-boew.github.io/themes-dist/GCWeb/GCWeb/assets/favicon.ico" rel="icon" type="image/x-icon" />
<link rel="stylesheet" href="https://wet-boew.github.io/themes-dist/GCWeb/GCWeb/css/theme.min.css" />
<noscript><link rel="stylesheet" href="https://wet-boew.github.io/themes-dist/GCWeb/wet-boew/css/noscript.min.css" /></noscript>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/2.2.4/jquery.js"></script>
<script src="https://wet-boew.github.io/themes-dist/GCWeb/wet-boew/js/wet-boew.min.js"></script>
<script src="https://wet-boew.github.io/themes-dist/GCWeb/GCWeb/js/theme.min.js"></script>
"""

LICENSE_HTML_EN = (
    "<p>licenced under the "
    "<a rel='license' href='https://open.canada.ca/en/open-government-licence-canada'>Open Government Licence – Canada 2.0</a>"
    "</p>"
)

LICENSE_HTML_FR = (
    "<p>information visée par la "
    "<a rel='license' href='https://ouvert.canada.ca/fr/licence-du-gouvernement-ouvert-canada'>Licence du gouvernement ouvert – Canada 2.0</a>"
    "</p>"
)

GENERAL_LABELS = {
    'en': {
        'name': 'Name of institution:',
        'reporting': 'Reporting period:',
        'to': 'to:',
        'page_details': 'Page details',
        'question': 'Question',
        'number': 'Number',
    },
    'fr': {
        'name': "Nom de l’institution :",
        'reporting': "Période d’établissement de rapport :",
        'to': 'à :',
        'page_details': 'Détails de la page',
        'question': 'Question',
        'number': 'Nombre',
    },
}

_SERVER = {'thread': None, 'server': None, 'port': None}


def extract_form_title(path: Path, locale: str) -> str:
    needle = 'Supplemental Statistical Report' if locale == 'en' else 'Rapport statistique supplémentaire'
    wb = openpyxl.load_workbook(path, data_only=True)
    ws = wb.active
    for row in ws.iter_rows():
        for cell in row:
            val = cell.value
            if isinstance(val, str) and needle in val:
                return val.strip()
    return needle


def extract_section_titles(path: Path) -> dict:
    wb = openpyxl.load_workbook(path, data_only=True)
    ws = wb.active
    titles = {}
    for row in ws.iter_rows():
        for cell in row:
            val = cell.value
            if isinstance(val, str):
                match = re.match(r'^\s*Section\s*(\d+)\s*:\s*(.*)', val)
                if match:
                    titles[match.group(1)] = match.group(2).strip()
    return titles


def find_subsection_row(ws, subsection_number: str):
    needle = f"{subsection_number} "
    for row in ws.iter_rows():
        for cell in row:
            val = cell.value
            if isinstance(val, str) and val.strip().startswith(needle):
                return cell.row
    return None


def build_table_layout(ws, subsection_number: str, row_count: int, col_count: int, labels: dict, locale: str, section_titles: dict):
    if subsection_number in ('3.0', '4.0'):
        section_number = subsection_number.split('.')[0]
        return {
            'caption': f"{subsection_number} {section_titles.get(section_number, '')}".strip(),
            'row_header': GENERAL_LABELS[locale]['question'],
            'col_headers': [GENERAL_LABELS[locale]['number']],
            'row_labels': [labels.get(subsection_number, '')],
        }

    start_row = find_subsection_row(ws, subsection_number)
    if start_row is None:
        return {
            'caption': subsection_number,
            'row_header': '',
            'col_headers': [''] * col_count,
            'row_labels': [f"Row {idx}" for idx in range(1, row_count + 1)],
        }

    header_row = None
    header_vals = None
    for r in range(start_row + 1, start_row + 15):
        row_vals = [ws.cell(row=r, column=c).value for c in range(1, ws.max_column + 1)]
        if row_vals[0] and any(v not in (None, '') for v in row_vals[1:]):
            header_row = r
            header_vals = row_vals
            break

    row_header = header_vals[0] if header_vals else ''
    col_headers = []
    if header_vals:
        for val in header_vals[1:]:
            if val not in (None, ''):
                col_headers.append(str(val).strip())

    if len(col_headers) < col_count:
        col_headers.extend([''] * (col_count - len(col_headers)))
    if len(col_headers) > col_count:
        col_headers = col_headers[:col_count]

    row_labels = []
    if header_row:
        r = header_row + 1
        while r <= ws.max_row and len(row_labels) < row_count:
            val = ws.cell(row=r, column=1).value
            if isinstance(val, str) and val.strip():
                if val.strip().startswith('Section'):
                    break
                if re.match(r'^\d+\.\d+', val.strip()):
                    break
                row_labels.append(val.strip())
            r += 1

    if len(row_labels) < row_count:
        for idx in range(len(row_labels) + 1, row_count + 1):
            row_labels.append(f"Row {idx}")

    return {
        'caption': f"{subsection_number} {labels.get(subsection_number, '')}".strip(),
        'row_header': str(row_header).strip() if row_header else '',
        'col_headers': col_headers,
        'row_labels': row_labels,
    }


def build_form_structure(locale: str):
    form_path = FORM_FILES[locale]
    key_df = load_key_sheet(form_path)

    records = []
    for _, row in key_df.iterrows():
        records.append({'id': row['id'], 'subsection_number': row['subsection_number']})
    expanded = expand_supplemental_ids(records)

    grid = {}
    for record in expanded:
        match = re.match(r'^Q(\d+)R(\d+)C(\d+)$', record['id'])
        if not match:
            continue
        q_num = int(match.group(1))
        row_num = int(match.group(2))
        col_num = int(match.group(3))
        info = grid.setdefault(q_num, {'rows': 0, 'cols': 0})
        info['rows'] = max(info['rows'], row_num)
        info['cols'] = max(info['cols'], col_num)

    sub_to_q = {}
    for _, row in key_df.iterrows():
        q_match = re.match(r'^Q(\d+)', row['id'])
        if q_match:
            sub_to_q[row['subsection_number']] = int(q_match.group(1))

    section_titles = extract_section_titles(form_path)
    labels = extract_subsection_labels(form_path, locale)
    title = extract_form_title(form_path, locale)

    wb = openpyxl.load_workbook(form_path, data_only=True)
    ws = wb.active

    def _sub_sort(val: str):
        return tuple(int(p) for p in val.split('.'))

    subsections = []
    for sub_num in sorted(sub_to_q.keys(), key=_sub_sort):
        q_num = sub_to_q[sub_num]
        rows = grid.get(q_num, {}).get('rows', 1)
        cols = grid.get(q_num, {}).get('cols', 1)
        layout = build_table_layout(ws, sub_num, rows, cols, labels, locale, section_titles)
        subsection = {
            'subsection_number': sub_num,
            'section_number': sub_num.split('.')[0],
            'q_num': q_num,
            'rows': rows,
            'cols': cols,
            **layout,
        }
        if sub_num in ('3.0', '4.0'):
            section_number = sub_num.split('.')[0]
            subsection['caption'] = f"{sub_num} {section_titles.get(section_number, '')}".strip()
        subsections.append(subsection)

    sections = {}
    for sub in subsections:
        sec_num = sub['section_number']
        sec_title = section_titles.get(sec_num, f"Section {sec_num}")
        section = sections.setdefault(sec_num, {'number': sec_num, 'title': sec_title, 'subsections': []})
        section['subsections'].append(sub)

    ordered_sections = [sections[k] for k in sorted(sections.keys(), key=int)]

    return {
        'title': title,
        'sections': ordered_sections,
    }


def build_table(doc, subsection):
    table = doc.new_tag('table')
    table['class'] = 'table table-bordered table-condensed small'

    caption = doc.new_tag('caption')
    caption['class'] = 'h3 text-left'
    caption.string = subsection['caption']
    table.append(caption)

    thead = doc.new_tag('thead')
    tr = doc.new_tag('tr')
    tr['class'] = 'active'

    th_label = doc.new_tag('th')
    th_label['class'] = 'col-xs-8'
    th_label['scope'] = 'col'
    th_label.string = subsection['row_header']
    tr.append(th_label)

    for col_header in subsection['col_headers']:
        th = doc.new_tag('th')
        th['class'] = 'text-center col-xs-4'
        th['scope'] = 'col'
        th.string = col_header
        tr.append(th)

    thead.append(tr)
    table.append(thead)

    tbody = doc.new_tag('tbody')
    for row_idx, row_label in enumerate(subsection['row_labels'], start=1):
        tr = doc.new_tag('tr')
        th = doc.new_tag('th')
        th['scope'] = 'row'
        th.string = row_label
        tr.append(th)

        for col_idx, col_header in enumerate(subsection['col_headers'], start=1):
            td = doc.new_tag('td')
            input_tag = doc.new_tag('input')
            input_tag['id'] = f"Q{subsection['q_num']}R{row_idx}C{col_idx}"
            input_tag['type'] = 'text'
            input_tag['class'] = 'center-block small'
            input_tag['size'] = '8'
            input_tag['maxlength'] = '10'
            title_parts = [subsection['caption'], row_label]
            if col_header:
                title_parts.append(col_header)
            input_tag['title'] = ' - '.join(title_parts)
            td.append(input_tag)
            tr.append(td)

        tbody.append(tr)

    table.append(tbody)
    return table


def build_form_soup(structure, locale: str):
    doc = BeautifulSoup('<!doctype html><html><head></head><body></body></html>', 'html.parser')
    doc.html['lang'] = 'fr' if locale == 'fr' else 'en'

    head = doc.head
    head.append(doc.new_tag('meta', charset='utf-8'))
    head.append(doc.new_tag('meta', attrs={'name': 'viewport', 'content': 'width=device-width,initial-scale=1'}))
    title_tag = doc.new_tag('title')
    title_tag.string = structure['title']
    head.append(title_tag)

    main = doc.new_tag('main')
    main['class'] = 'container'
    main['property'] = 'mainContentOfPage'
    main['resource'] = '#wb-main'
    main['typeof'] = 'WebPageElement'

    title_wrap = doc.new_tag('div')
    title_wrap['class'] = 'mwstitle section'
    h1 = doc.new_tag('h1')
    h1['id'] = 'wb-cont'
    h1['property'] = 'name'
    h1.string = structure['title']
    title_wrap.append(h1)
    main.append(title_wrap)

    general = doc.new_tag('div')
    general['class'] = 'mwsgeneric-base-html parbase section'
    general_inner = doc.new_tag('div')
    general_inner['class'] = 'small'

    name_p = doc.new_tag('p')
    name_strong = doc.new_tag('strong')
    name_label = doc.new_tag('label', **{'for': 'NameOfInstitution'})
    name_label.string = GENERAL_LABELS[locale]['name']
    name_strong.append(name_label)
    name_p.append(name_strong)
    name_input = doc.new_tag('input', id='NameOfInstitution', type='text')
    name_p.append(name_input)
    general_inner.append(name_p)

    fieldset = doc.new_tag('fieldset')
    fieldset['class'] = 'brdr-bttm'
    legend = doc.new_tag('legend')
    legend['class'] = 'wb-inv'
    legend_strong = doc.new_tag('strong')
    legend_strong.string = GENERAL_LABELS[locale]['reporting']
    legend.append(legend_strong)
    fieldset.append(legend)

    row_section = doc.new_tag('section')
    row_section['class'] = 'row'

    col_start = doc.new_tag('div')
    col_start['class'] = 'col-xs-8'
    label_start = doc.new_tag('label', **{'for': 'ReportingPeriodStart'})
    label_start.string = GENERAL_LABELS[locale]['reporting']
    col_start.append(label_start)
    input_start = doc.new_tag('input', id='ReportingPeriodStart', type='text', size='10')
    col_start.append(input_start)

    col_end = doc.new_tag('div')
    col_end['class'] = 'col-xs-4'
    label_end = doc.new_tag('label', **{'for': 'ReportingPeriodEnd'})
    label_end.string = GENERAL_LABELS[locale]['to']
    col_end.append(label_end)
    input_end = doc.new_tag('input', id='ReportingPeriodEnd', type='text', size='10')
    col_end.append(input_end)

    row_section.append(col_start)
    row_section.append(col_end)
    fieldset.append(row_section)
    fieldset.append(doc.new_tag('div', **{'class': 'clearfix'}))
    fieldset.append(doc.new_tag('hr'))

    general_inner.append(fieldset)
    general.append(general_inner)
    main.append(general)

    for section in structure['sections']:
        sec = doc.new_tag('section')
        sec['class'] = 'panel panel-info'

        header = doc.new_tag('header')
        header['class'] = 'panel-heading'
        h2 = doc.new_tag('h2')
        h2['class'] = 'mrgn-tp-0 mrgn-bttm-0'
        h2.string = f"Section {section['number']}: {section['title']}"
        header.append(h2)
        sec.append(header)

        body = doc.new_tag('div')
        body['class'] = 'panel-body'

        for subsection in section['subsections']:
            table_wrap = doc.new_tag('div')
            table_wrap['class'] = 'table-responsive'
            table_wrap.append(build_table(doc, subsection))
            body.append(table_wrap)
            body.append(doc.new_tag('div', **{'class': 'clearfix'}))

        sec.append(body)
        main.append(sec)

    pagedetails = doc.new_tag('section', id='pagedetails')
    pagedetails['class'] = 'pagedetails'
    h2 = doc.new_tag('h2')
    h2['class'] = 'wb-inv'
    h2.string = GENERAL_LABELS[locale]['page_details']
    pagedetails.append(h2)
    date_mod = doc.new_tag('gcds-date-modified')
    date_mod.string = pd.Timestamp.today().date().isoformat()
    pagedetails.append(date_mod)
    main.append(pagedetails)

    doc.body.append(main)
    return doc


def apply_wet_assets(doc):
    head = doc.head or doc
    for tag in BeautifulSoup(WET_HEAD_HTML, 'html.parser').find_all(['link', 'script', 'noscript']):
        head.append(tag)
    return doc


def add_license_footer(doc, lang):
    html = LICENSE_HTML_FR if str(lang).lower().startswith('fr') else LICENSE_HTML_EN
    block = BeautifulSoup(html, 'html.parser')
    pagedetails = doc.find(id='pagedetails')
    if pagedetails:
        date_mod = pagedetails.find('gcds-date-modified')
        if date_mod:
            date_mod.insert_after(block)
        else:
            pagedetails.append(block)
    else:
        main = doc.find('main')
        if main and main.parent:
            main.append(block)
        else:
            doc.body.append(block)
    return doc


def populate_form(soup, values):
    for input_tag in soup.find_all('input'):
        input_id = input_tag.get('id')
        if not input_id:
            continue
        if input_id in values:
            val = values[input_id]
            if pd.isna(val):
                continue
            input_tag['value'] = str(val)
    return soup


def clean_html_assets(html_text: str) -> str:
    return html_text


def load_data(csv_path='Supplemental-AI_refactored.csv'):
    df = pd.read_csv(csv_path)
    df['ReportingPeriodStart'] = pd.to_datetime(df['ReportingPeriodStart'])
    return df


def build_form_structures():
    return {
        'en': build_form_structure('en'),
        'fr': build_form_structure('fr'),
    }


FORM_STRUCTURES = build_form_structures()


def generate_populated_html(df, gc_org_id, start_date, lang='En', output_html='populated_supplemental_form_example.html'):
    subset = df[(df['gc_orgID'] == gc_org_id) & (df['ReportingPeriodStart'] == pd.to_datetime(start_date))]
    if subset.empty:
        raise ValueError('No rows found for selection')
    values = dict(zip(subset['id'], subset['value']))

    first_row = subset.iloc[0]
    inst_name = first_row['institution_fr'] if str(lang).lower().startswith('fr') else first_row['institution_en']
    values['NameOfInstitution'] = inst_name
    start_dt = pd.to_datetime(first_row['ReportingPeriodStart']).date()
    end_dt = pd.to_datetime(first_row['ReportingPeriodEnd']).date() if pd.notna(first_row['ReportingPeriodEnd']) else None
    values['ReportingPeriodStart'] = start_dt.isoformat()
    if end_dt:
        values['ReportingPeriodEnd'] = end_dt.isoformat()

    locale = 'fr' if str(lang).lower().startswith('fr') else 'en'
    soup = build_form_soup(FORM_STRUCTURES[locale], locale)
    soup = populate_form(soup, values)
    soup = apply_wet_assets(soup)
    soup = add_license_footer(soup, locale)
    html_text = soup.encode(formatter=None).decode('utf-8')
    html_text = clean_html_assets(html_text)
    output_path = Path(output_html)
    output_path.write_text(html_text, encoding='utf-8')
    return output_path


def _find_port(preferred=8000):
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('localhost', preferred))
            return preferred
    except OSError:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('localhost', 0))
            return s.getsockname()[1]


def ensure_server(root='.', port=8000):
    if _SERVER['thread'] and _SERVER['thread'].is_alive():
        return _SERVER['port']
    chosen_port = _find_port(port)
    handler = functools.partial(SimpleHTTPRequestHandler, directory=str(Path(root).resolve()))
    server = ThreadingHTTPServer(('localhost', chosen_port), handler)
    thread = threading.Thread(target=server.serve_forever, daemon=True)
    thread.start()
    _SERVER.update({'thread': thread, 'server': server, 'port': chosen_port})
    return chosen_port


def open_local_page(html_path, port=8000):
    port = ensure_server(root=Path(html_path).parent, port=port)
    url = f"http://localhost:{port}/{Path(html_path).name}"
    webbrowser.open(url)
    return url

# Load dataset and prepare widgets

DATA_CSV = 'Supplemental-AI_refactored.csv'
data_df = load_data(DATA_CSV)

institution_options = sorted(
    data_df[['gc_orgID','institution_en','institution_fr']].drop_duplicates().itertuples(index=False),
    key=lambda r: (r.gc_orgID, r.institution_en)
)
inst_dropdown = widgets.Dropdown(
    options=[(f"{r.gc_orgID} | {r.institution_en}", r.gc_orgID) for r in institution_options],
    description='Institution',
    layout=widgets.Layout(width='80%')
)
period_options = sorted(data_df['ReportingPeriodStart'].dropna().dt.date.unique())
period_dropdown = widgets.Dropdown(options=period_options, description='Period')
lang_dropdown = widgets.Dropdown(options=['En','Fr'], description='Language')
generate_btn = widgets.Button(description='Generate HTML', button_style='primary', disabled=True)
status_out = widgets.Output()

def on_change(_):
    generate_btn.disabled = not (inst_dropdown.value and period_dropdown.value and lang_dropdown.value)

inst_dropdown.observe(on_change, names='value')
period_dropdown.observe(on_change, names='value')
lang_dropdown.observe(on_change, names='value')

def on_click(_):
    status_out.clear_output()
    with status_out:
        try:
            output_path = generate_populated_html(
                data_df,
                inst_dropdown.value,
                period_dropdown.value,
                lang_dropdown.value,
                output_html='populated_supplemental_form_example.html',
            )
            url = open_local_page(output_path)
            display(HTML(f"Saved to <code>{output_path}</code> — Serving locally: <a href='{url}' target='_blank'>{url}</a>"))
            display(HTML(f"<iframe src='{url}' width='100%' height='800px'></iframe>"))
        except Exception as e:
            print('Error:', e)

generate_btn.on_click(on_click)
on_change(None)

ui = widgets.VBox([inst_dropdown, period_dropdown, lang_dropdown, generate_btn, status_out])
display(ui)
